# Getting started

Sertit utils is a set of functions for geo analysis. Blablabla

## Open a raster image

In [ ]:
from sertit import AnyPath, rasters
from sertit.unistra import unistra_s3

with unistra_s3():
    pth = AnyPath("s3://sertit-sertit-utils-ci/tutorials/getting_started/image.tif")
    r = rasters.read(pth)

For the moment, let ignore the unistra_s3() call since you don't need it to read local images, we'll talk about it later. 

What happened in this code ?
We initialize a geotiff file with `AnyPath` and then read it thanks to `rasters.read`. What is the return type of this function ?


In [ ]:
r[0:2, :, :]

In [ ]:
r[0:3, ::10, ::10].plot.imshow(robust=True)

In [ ]:
r.data

The return type in `xarray.DataArray`. 
`xarray` is a python library to work with labelled multi-dimensional arrays.
Actually, `sertit` use rioxarray to open your raster. `rioxarray` extends `xarray` with the `rio` accessor. Nice. What does it mean ? Simply that you can read and process geographic images thanks to `rioxarray` while `xarray` only understand "classical" images. 

Then what is a geographic DataArray ? It's a containers with data, coordinates and attributes. Let's print them all.

In [ ]:
# Print x and y coordinates
print("X coordinates:\n", r.x)
print("\n\nY ccordinates: ", r.y)

Coordinates are themselfs DataArray ! What about the data and spatial_ref ? Spatial ref can be accessed with `r.spatial_ref`. For data, it's a little bit tricky because `rioxarray` does not print it by default. But you can access it with the `data` attribute. `data` is a dask array containing the real value of our pixels in the image. Our data array only contains one band.

In [ ]:
# print the value of the pixels in the image
r.data

In [ ]:
# The spatial reference is stored in the coordinates of our raster and not in attributes !
r.spatial_ref

You probably noticed that coordinates rasters contain coordinates in the CRS of the 
image. In GeoTiff, coordinated are stored in a matrice whose origin is (0,0). Thanks to spatial_ref and to the vector GeoTransform, rioxarray is able to convert the coordinates centered in (0,0) to coordinates in CRS image.

In [ ]:
# So finally what are the attributes of our array ?
r.attrs
# Ok not really interisting...

## Crop the image

Sertit library offers a ready to use function to crop a raster by an AOI. 
First we use vectors from sertit which can read vectors from any type of data (shapefile, kml...) and return a GeoDtaFrame.

In [ ]:
from sertit import vectors

with unistra_s3():
    aoi_path = AnyPath("s3://sertit-sertit-utils-ci/tutorials/getting_started/aoi.shp")
    aoi = vectors.read(aoi_path)

In [ ]:
aoi

In [ ]:
aoi.explore()

In [ ]:
print(type(aoi))

We can use any functions from geopandas, for example esimate_crs

In [ ]:
print("Estimate CRS is: ", aoi.estimate_utm_crs())

In [ ]:
crop_image = rasters.crop(r, aoi)
crop_image[0:3, ::10, ::10].plot.imshow(robust=True)

Then we can write the output thanks to `rasters.write`. This function is powerful enough to handle compression automatically and write big images (see notebook about processing big images).

In [ ]:
rasters.write(crop_image, "crop_image.tif")

## Reprojection

## Custom process

Sometimes sertit utils does not contain the function you need. You can use odc-geo (for reporjection for example) or rioxarray. But it could be not enough again. It's where apply_func comes in handy.

Let's say we want to classify our raster with the following condition:
- If pixel < 6.7 : pixel=1
- If 6.7 <= pixel < 11.2 : pixel=2
- If 11.2 <= pixel < 22.4 : pixel=3
- If 22.4 <= pixel < 33.6 : pixel=4
- If pixel >= 33.6 : pixel=5

We can use xarray to proceed. Never convert your array to numpy array !

In [ ]:
import xarray

from sertit import AnyPath, rasters
from sertit.unistra import unistra_s3

with unistra_s3():
    pth = AnyPath(
        "s3://sertit-sertit-utils-ci/tutorials/getting_started/MeanSoilLoss.tif"
    )
    r = rasters.read(pth)

conditions = [
    (r.data < 6.7),
    (r.data >= 6.7) & (r.data < 11.2),
    (r.data >= 11.2) & (r.data < 22.4),
    (r.data >= 22.4) & (r.data < 33.6),
    (r.data >= 33.6),
]
for i, condition in enumerate(conditions):
    r.data = xarray.where(condition, i + 1, r.data)

In [ ]:
# dask array processes are lazy. The cell above does not perform anything.
# We can explicitely call compute() to load the result in memory
# but some methods implicitely call it (plot, write)
# r = r.compute()

In [ ]:
r

In [ ]:
# The plot method actually calls compute()
r.plot()